In [16]:
%matplotlib inline
import numpy as np
import torch
import torch.optim as optim

torch.set_printoptions(edgeitems=2, linewidth=75)

In [17]:
t_c = [0.5,  14.0, 15.0, 28.0, 11.0,  8.0,  3.0, -4.0,  6.0, 13.0, 21.0]
t_u = [35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4]
t_c = torch.tensor(t_c).unsqueeze(1) # <1>
t_u = torch.tensor(t_u).unsqueeze(1) # <1>

t_u.shape

torch.Size([11, 1])

In [18]:
n_samples = t_u.shape[0]
n_val = int(0.2 * n_samples)

shuffled_indices = torch.randperm(n_samples)

train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]

train_indices, val_indices

(tensor([ 2,  1,  5,  0,  3,  9,  8,  6, 10]), tensor([4, 7]))

In [19]:
t_u_train = t_u[train_indices]
t_c_train = t_c[train_indices]

t_u_val = t_u[val_indices]
t_c_val = t_c[val_indices]

t_un_train = 0.1 * t_u_train
t_un_val = 0.1 * t_u_val


In [21]:
import torch.nn as nn

linear_model = nn.Linear(1, 1) # args: input size, output size, bias defaulting to True
linear_model(t_un_val)

tensor([[-2.1144],
        [-0.2125]], grad_fn=<AddmmBackward0>)

In [22]:
linear_model.weight

Parameter containing:
tensor([[-0.5513]], requires_grad=True)

In [24]:
linear_model.bias

Parameter containing:
tensor([0.9892], requires_grad=True)

In [25]:
x = torch.ones(1)
linear_model(x)

tensor([0.4379], grad_fn=<ViewBackward0>)

In [26]:
x = torch.ones(10, 1)
linear_model(x)

tensor([[0.4379],
        [0.4379],
        [0.4379],
        [0.4379],
        [0.4379],
        [0.4379],
        [0.4379],
        [0.4379],
        [0.4379],
        [0.4379]], grad_fn=<AddmmBackward0>)

In [27]:
linear_model = nn.Linear(1, 1)
optimizer = optim.SGD(
linear_model.parameters(),
lr=1e-2)

In [28]:
linear_model.parameters()

<generator object Module.parameters at 0x723240db75a0>

In [29]:
list(linear_model.parameters())

[Parameter containing:
 tensor([[-0.7853]], requires_grad=True),
 Parameter containing:
 tensor([-0.3143], requires_grad=True)]

In [ ]:
def training_loop(n_epochs, optimizer, model, loss_fn, t_u_train, t_u_val,
                  t_c_train, t_c_val):
    for epoch in range(1, n_epochs + 1):
        t_p_train = model(t_u_train)
        loss_train = loss_fn(t_p_train, t_c_train)

        t_p_val = model(t_u_val)
        loss_val = loss_fn(t_p_val, t_c_val)

        optimizer.zero_grad()
        loss_train.backward()
        optimizer.step()

        if epoch == 1 or epoch % 1000 == 0:
            print(f"Epoch {epoch}, Training loss {loss_train.item():.4f},"
                  f" Validation loss {loss_val.item():.4f}")